In [3]:
import pandas as pd

# Read data from files
train = pd.read_csv('labeledTrainData.tsv', header=0, delimiter='\t', quoting=3)
test = pd.read_csv('testData.tsv', header=0, delimiter='\t', quoting=3)
unlabeled_train = pd.read_csv('unlabeledTrainData.tsv', header=0, delimiter='\t', quoting=3)

# Verify the numer of reviews that were read (100, 00 in total)
print("""Read {} labeled train reviews, 
{} labeled test reviews, 
and {} unlabeled reviews""".format(
    train['review'].size, test['review'].size, unlabeled_train['review'].size))

Read 25000 labeled train reviews, 
        25000 labeled test reviews, 
        and 50000 unlabeled reviews


In [4]:
# Import various modules for string cleaning
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords

def review_to_wordlist(review, remove_stopwords=False):
    """Function to convert a  document to a sequence of words, optionally removing stop words. Returns a list of words."""
    # 1. Remove HTML
    review_text = BeautifulSoup(review).get_text()
    # 2. Remove Non-letters
    review_text = re.sub('[^a-zA-Z]', ' ', review_text)
    # 3. Convert words to lower case and split them
    words = review_text.lower().split()
    # 4. Optionally remove stop words
    if remove_stopwords:
        stops = set(stopwords.words('english'))
        words = [word for word in words if not word in stops]
    return words
        

In [8]:
# Downlaod the punkt tokenizer for sentence splitting
import nltk.data
nltk.download()

# Load the punkt tokenizer
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

# Define a function ot split a review into parsed sentences
def review_to_sentences(review, tokenzier, remove_stopwords=False):
    """Function to split a review into parsed sentences. Returns
    a lsit of sentences, where each sentence is a list of words"""
    # 1. User the NLTK tokenizer to split the paragraph into sentences
    raw_sentences = tokenizer.tokenize(review.strip())
    # 2. Loop over each sentence
    sentences = []
    for raw_sentence in raw_sentences:
        #If a sentence is empty, skip it
        if len(raw_sentence) > 0:
            # Otherwise, call review_to_wordlist to get a list of words
            sentences.append(review_to_wordlist(raw_sentence, remove_stopwords))
    # Return the list of sentences (each sentence is a list of words,
    # so this returns a list of lists)
    return sentences

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


In [10]:
sentences = [] # Initialize an empty list of sentences

print('Parsing sentences from training set')
for review in train['review']:
    sentences += review_to_sentences(review, tokenizer)
    
print('Parsing sentences from unlabled set')
for review in unlabeled_train['review']:
    sentences += review_to_sentences(review, tokenizer)

Parsing sentences from training set


/home/memery/anaconda3/lib/python3.5/site-packages/bs4/__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))
/home/memery/anaconda3/lib/python3.5/site-packages/bs4/__init__.py:198: UserWarning: "b'.'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  '"%s" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.' % markup)
/home/memery/anaconda3/lib/python3.5/site-packages/bs4/__init__.py:207: UserWarning: "b'http://www.happierabroad.com"'" looks like a URL. Beautiful So

Parsing sentences from unlabled set


/home/memery/anaconda3/lib/python3.5/site-packages/bs4/__init__.py:207: UserWarning: "b'http://www.archive.org/details/LovefromaStranger"'" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.
  '"%s" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.' % markup)
/home/memery/anaconda3/lib/python3.5/site-packages/bs4/__init__.py:207: UserWarning: "b'http://www.loosechangeguide.com/LooseChangeGuide.html"'" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.
  '"%s" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful So

In [11]:
print(len(sentences))
print(sentences[0])
print(sentences[1])

795538
['with', 'all', 'this', 'stuff', 'going', 'down', 'at', 'the', 'moment', 'with', 'mj', 'i', 've', 'started', 'listening', 'to', 'his', 'music', 'watching', 'the', 'odd', 'documentary', 'here', 'and', 'there', 'watched', 'the', 'wiz', 'and', 'watched', 'moonwalker', 'again']
['maybe', 'i', 'just', 'want', 'to', 'get', 'a', 'certain', 'insight', 'into', 'this', 'guy', 'who', 'i', 'thought', 'was', 'really', 'cool', 'in', 'the', 'eighties', 'just', 'to', 'maybe', 'make', 'up', 'my', 'mind', 'whether', 'he', 'is', 'guilty', 'or', 'innocent']


In [15]:
# Import the built-in logging module and configure it so that Word2Vec
# creates nice ouput messages
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

# Set values for various parameters
num_features = 300 # Word vector dimensionality
min_word_count = 40 # Minimum word count
num_workers = 4 # Number of threads to run in parallel
context = 10 # Context window size
downsampling = 1e-3 # Downsample setting for frequence words

# Initialize and train model (this will take some time)
from gensim.models import word2vec
print('Training model...')
model = word2vec.Word2Vec(sentences,
                         workers=num_workers,
                         size=num_features,
                         min_count=min_word_count,
                         window=context,
                         sample=downsampling)
# If you don't plan to train the model any further, calling
# init_sims will make the model much more memory-efficient.
model.init_sims(replace=True)
# It can be helpful to create a meaningful model name and
# save the model for later use. You can load it later using Word2Vec.load()
model_name = '300features_40minwords_10context'
model.save(model_name)

Training model...


In [16]:
print(model.doesnt_match('man woman child kitchen'.split()))

kitchen


In [17]:
print(model.doesnt_match('france england germany berlin'.split()))

berlin


In [18]:
print(model.doesnt_match('paris berlin london austria'.split()))

paris


In [20]:
print(model.most_similar('man'))
print(model.most_similar('queen'))

[('woman', 0.620460033416748), ('lady', 0.5978381633758545), ('lad', 0.5947127342224121), ('millionaire', 0.5336859226226807), ('guy', 0.5209099054336548), ('monk', 0.5120577812194824), ('men', 0.5044459104537964), ('soldier', 0.5028200149536133), ('boy', 0.49186182022094727), ('person', 0.49071723222732544)]
[('princess', 0.6938101649284363), ('bride', 0.6378064155578613), ('maid', 0.5931621789932251), ('belle', 0.5840067267417908), ('stepmother', 0.5785546898841858), ('mistress', 0.5769913792610168), ('nun', 0.5756708383560181), ('mary', 0.5738168954849243), ('goddess', 0.5737065076828003), ('victoria', 0.5731890201568604)]


In [21]:
print(model.most_similar('awful'))

[('terrible', 0.7707134485244751), ('horrible', 0.7411921620368958), ('atrocious', 0.7342128753662109), ('abysmal', 0.7259578704833984), ('dreadful', 0.703248918056488), ('appalling', 0.6884777545928955), ('horrendous', 0.6792418956756592), ('horrid', 0.6619541049003601), ('lousy', 0.6417896747589111), ('amateurish', 0.6281046867370605)]


In [22]:
# Load the model that we created in Part 2
from gensim.models import Word2Vec
model = Word2Vec.load('300features_40minwords_10context')
print(type(model.syn0))
print(model.syn0.shape)

<class 'numpy.ndarray'>
(16490, 300)


In [23]:
print(model['flowers'])

[ 0.2618922   0.00607259 -0.02737801  0.06234417 -0.04600378 -0.01870163
  0.01574566  0.06589511  0.03533733 -0.02260944  0.11126167  0.03386962
 -0.0160492  -0.12030359  0.03212173 -0.03332198 -0.01643397 -0.04414876
  0.06900069 -0.01394662  0.0328128  -0.02321619  0.05129942 -0.00926392
 -0.01397796 -0.00944078  0.05401015  0.02342801  0.07461627  0.00282747
  0.06374865 -0.05665589  0.07150409 -0.03846945  0.0469505  -0.01519634
  0.02555021 -0.01127653 -0.06053138  0.03177625 -0.03894054 -0.02675777
  0.10457046 -0.13731593  0.01265697 -0.0304191   0.13601707 -0.04821696
 -0.05875409  0.05285454  0.00799542 -0.09708098 -0.0089174  -0.01670512
 -0.00739111  0.03731174 -0.02266014 -0.03806334 -0.04263867 -0.05053771
 -0.08273013 -0.05926327  0.10405719 -0.02973797  0.07222268 -0.01441816
 -0.07967248  0.05233915 -0.0727954   0.01434353 -0.00204398 -0.05825311
 -0.01953697  0.07926004 -0.11219627 -0.02612725 -0.06892834 -0.02042108
  0.00042455 -0.06259683  0.03985948  0.11488532 -0

In [24]:
import numpy as np

def makeFeatureVec(words, model, num_features):
    """Function to average all the word vectors in a given paragraph"""
    # Pre-initialize an empty numpy array (for speed)
    featureVec = np.zeros((num_features,),dtype='float32')
    nwords = 0
    # Index2word is a list that contains the names of the words in
    # the model's vocabulary. Convert it to a set, for speed
    index2word_set = set(model.index2word)
    
    # Loop over each word in the review and, if it is in the model's
    # vocabulary, add it's feature vector to the total
    for word in words:
        if word in index2word_set:
            nwords = nwords + 1
            featureVec = np.add(featureVec, model[word])
    
    # divide the result by the number of words to get the average
    featureVec = np.divide(featureVec, nwords)
    return featureVec


def getAvgFeatureVecs(reviews, model, num_features):
    """Given a set of reviews (each one a list of words), calculate
     the average feature vector for each one and return a 2D numpy array"""
    # Initialize a counter
    counter = 0.
    # Preallocate a 2D numpy array, for speed
    reviewFeatureVecs = np.zeros((len(reviews), num_features), dtype='float32')
    # Loop through the reviews
    
    for review in reviews:
        # Print a status message every 1000th review
        if counter % 1000. == 0.:
            print('Review {} of {}'.format(counter, len(reviews)))
            
        # Call the function (defined above) that makes average feature vectors
        reviewFeatureVecs[counter] = makeFeatureVec(review, model, num_features)
        #Increment the counter
        counter += 1
    return reviewFeatureVecs
        

In [26]:
clean_train_reviews = []
for review in train['review']:
    clean_train_reviews.append(review_to_wordlist(review, remove_stopwords=True))
    
trainDataVecs = getAvgFeatureVecs(clean_train_reviews, model, num_features)
print('Creating average feature vecs for test reviews')
clean_test_reviews = []
for review in test['review']:
    clean_test_reviews.append(review_to_wordlist(review, remove_stopwords=True))
    
testDataVecs = getAvgFeatureVecs(clean_test_reviews, model, num_features)

/home/memery/anaconda3/lib/python3.5/site-packages/bs4/__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


Review 0.0 of 25000


/home/memery/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:39: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


Review 1000.0 of 25000
Review 2000.0 of 25000
Review 3000.0 of 25000
Review 4000.0 of 25000
Review 5000.0 of 25000
Review 6000.0 of 25000
Review 7000.0 of 25000
Review 8000.0 of 25000
Review 9000.0 of 25000
Review 10000.0 of 25000
Review 11000.0 of 25000
Review 12000.0 of 25000
Review 13000.0 of 25000
Review 14000.0 of 25000
Review 15000.0 of 25000
Review 16000.0 of 25000
Review 17000.0 of 25000
Review 18000.0 of 25000
Review 19000.0 of 25000
Review 20000.0 of 25000
Review 21000.0 of 25000
Review 22000.0 of 25000
Review 23000.0 of 25000
Review 24000.0 of 25000
Creating average feature vecs for test reviews
Review 0.0 of 25000
Review 1000.0 of 25000
Review 2000.0 of 25000
Review 3000.0 of 25000
Review 4000.0 of 25000
Review 5000.0 of 25000
Review 6000.0 of 25000
Review 7000.0 of 25000
Review 8000.0 of 25000
Review 9000.0 of 25000
Review 10000.0 of 25000
Review 11000.0 of 25000
Review 12000.0 of 25000
Review 13000.0 of 25000
Review 14000.0 of 25000
Review 15000.0 of 25000
Review 16000.0 